# Análisis Cluster: Crimen en USA

En  este notebook vamos a proceder al cálculo de clusters para el dataset con información del crimen en USA. En este caso no nos apoyaremos en el cálculo de Componentes Principales hechos en el notebook anterior para generar dichos clusters, si no que los construiremos directamente sobre los datos.

<div>
<img src="./media/crime.jpeg" width="500"/>
</div>

Comenzamos importando los módulos:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Cargamos los datos en un dataframe:

In [ ]:
raw_data = pd.read_csv('./data/USArrests.csv', index_col = 0)

Como siempre nos aseguramos de que los datos se hayan cargado correctamente:

In [ ]:
raw_data.head()

In [ ]:
raw_data.shape

De nuevo observamos que existen muchas diferencias entre las escalas de las variables por lo que pasamos como siempre a escalar los datos:

In [ ]:
from sklearn.preprocessing import scale
crime_data = pd.DataFrame(scale(raw_data), index=raw_data.index, columns=raw_data.columns)

Con esto hemos generado nuestros datos escalados:

In [ ]:
crime_data.head()

Una vez hecho esto podemos comenzar el cálculo de los clusters.
## Cluster no jerárquico

Comenzamos construyendo nuestro gráfico de sedimentación que nos ayudará a determinar el número de clusters que deseamos extraer:

In [ ]:
from sklearn.cluster import KMeans
ssd = []
for num_clusters in list(range(1,10)):
    model_clus = KMeans(n_clusters = num_clusters, max_iter=50)
    model_clus.fit(crime_data)
    ssd.append(model_clus.inertia_)

plt.plot(ssd)
plt.show()

La curva nos indica que pueden ser interesantes 1 o 3 clusters. Un cluster no aporta ningún tipo de información por lo que nos quedaremos con tres:

In [ ]:
numero_clusters = 3

Instanciamos el modelo pasando como parámetros el número de clusters, el máximo de iteraciones y una semilla aleatoria:

In [ ]:
cluster_model = KMeans(n_clusters = numero_clusters, max_iter=50,random_state = 50)

Una vez instanciado el modelo lo entrenamos con nuestros datos para obtener las etiquetas que asignan cada observación a un cluster:

In [ ]:
cluster_model.fit(crime_data)

In [ ]:
pd.Series(cluster_model.labels_)

Una vez tenemos las etiquetas las unimos con nuestro dataframe inicial para poder explorar los clusters y sacar conclusiones a partir de los mismos:

In [ ]:
df_no_index = raw_data

In [ ]:
df_no_index.reset_index(level=0, inplace=True)

Obtenemos el siguiente dataframe que asigna a cada estado sus métricas y su cluster resultante:

In [ ]:
clustered_components = pd.concat([df_no_index, pd.Series(cluster_model.labels_)], axis=1)

In [ ]:
cluster_no_jerar = clustered_components.set_index('index').rename(columns={0:'Cluster'})

In [ ]:
cluster_no_jerar

Exploramos ahora cada uno de los clusters para intentar interpretarlos:

In [ ]:
cluster_no_jerar[cluster_no_jerar.Cluster==0].head(20)

Parecen a simple vistas estados bastante neutros, podríamos pensar que son estados con una alta urbanización.

In [ ]:
cluster_no_jerar[cluster_no_jerar.Cluster==1].head(20)

Parece que nos encontramos ante los estados con mayor índice de criminalidad, se observa especialmente lo elevadas que son las cifras de asalto superando los 200 casos en la mayoría de los estados.

In [ ]:
cluster_no_jerar[cluster_no_jerar.Cluster==2].head(20)

En este último observamos los estados con menores índices de criminalidad en todos los aspectos de la misma (asalto, aseinato y violación).

### Explorando los clusters

Para asegurarnos que lo que teorizamos previamente es cierto calculamos las medias agregadas de cada cluster:

In [ ]:
Cluster_Murder=pd.DataFrame(cluster_no_jerar.groupby(["Cluster"]).Murder.mean())
Cluster_Assault=pd.DataFrame(cluster_no_jerar.groupby(["Cluster"]).Assault.mean())
Cluster_UrbanPop=pd.DataFrame(cluster_no_jerar.groupby(["Cluster"]).UrbanPop.mean())
Cluster_Rape=pd.DataFrame(cluster_no_jerar.groupby(["Cluster"]).Rape.mean())

In [ ]:
aggregated_mean = pd.concat([Cluster_Murder,Cluster_Assault,Cluster_UrbanPop,Cluster_Rape], axis=1)

In [ ]:
aggregated_mean

Observamos que en el cluster 0 tenemos los estados intermedios. Son además los estados más urbanizados aunque en este aspecto están muy próximos a los estados del cluster 1.

In [ ]:
estados_intermedios = cluster_no_jerar[cluster_no_jerar.Cluster==0]

El cluster 1 presenta los datos más alarmantes respecto a criminalidad en todos sus aspectos. Además su nivel de urbanización es también bastante elevado.

In [ ]:
estados_violentos = cluster_no_jerar[cluster_no_jerar.Cluster==1]

En el cluster 2 se  encuentran los estados con mejores datos en los que se reducen sobre todo las cifras de asalto y violación. Esto puede deberse en parte a la disminución de la urbanización de los estados pues estos crímenes suelen producirse más habitualmente en las ciudades.

In [ ]:
estados_pacificos = cluster_no_jerar[cluster_no_jerar.Cluster==2]

Podemos emplear gráficas de barras para visualizar la certitud de estas afirmaciones:

In [ ]:
fig = plt.figure(figsize = (10,6))
aggregated_mean.rename(index={0: 'Estados moderados'},inplace=True)
aggregated_mean.rename(index={2: 'Estados pacíficos'},inplace=True)
aggregated_mean.rename(index={1: 'Estados con más violencia'},inplace=True)

s=sns.barplot(x=aggregated_mean.index,y='Murder',data=aggregated_mean)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Tasa de asesinatos', fontsize=10)
plt.title('Estados en base al asesinato')
plt.show()


In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x=aggregated_mean.index,y='Assault',data=aggregated_mean)
plt.xlabel('Estados', fontsize=10)
plt.title('Estados en base al asalto')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x=aggregated_mean.index,y='Rape',data=aggregated_mean)
plt.xlabel('Estados', fontsize=10)
plt.title('Estados en base a la violación')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x=aggregated_mean.index,y='UrbanPop',data=aggregated_mean)
plt.xlabel('Estados', fontsize=10)
plt.title('Estados en base a la población')
plt.show()

Los gráficos de cajas pueden ser una buena referencia también para la obtención de conclusiones:

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='Cluster',y='Rape',data=cluster_no_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Número de violaciones', fontsize=10)
plt.title('Violación según el grupo')
plt.show()

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='Cluster',y='Assault',data=cluster_no_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Número de asaltos', fontsize=10)
plt.title('Asalto según el grupo')
plt.show()

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='Cluster',y='Murder',data=cluster_no_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Número de asesiantos', fontsize=10)
plt.title('Asesinatos según el grupo')
plt.show()

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='Cluster',y='UrbanPop',data=cluster_no_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Población', fontsize=10)
plt.title('Población según el grupo')
plt.show()

Podemos además explorar los datos relativos a una variable dentro de un mismo cluster:

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=estados_intermedios.index,y='Murder',data=estados_intermedios)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('Estado', fontsize=10)
plt.ylabel('Asesinatos', fontsize=10)
plt.title('Asesinatos en los estados moderados')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=estados_violentos.index,y='Murder',data=estados_violentos)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('Estado', fontsize=10)
plt.ylabel('Asesinatos', fontsize=10)
plt.title('Asesinatos en los estados violentos')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=estados_pacificos.index,y='Murder',data=estados_pacificos)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('Estado', fontsize=10)
plt.ylabel('Asesinatos', fontsize=10)
plt.title('Asesinatos en los estados seguros')
plt.show()

Una vez construidos los clusters desde una perspectiva no jerárquica podemos realizar la comparación con su obtención desde  un clustering jerárquico.

## Clustering jerárquico

Retomamos nuestros datos para el clustering, recordemos que en este caso estamos utilizando los datos escalados y no las componentes principales:

In [ ]:
crime_data.head()

Comenzamos construyendo el dendograma al que le pasamos el tipo de linkage: simple y el tipo de distancia: euclídea. 

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram


mergings_average=linkage(crime_data,method='single',metric='euclidean')
fig = plt.figure(figsize = (34,15))
dendrogram(mergings_average)
plt.show()

El dendograma parece genear clusters muy irregulares (siempre deja a un país aislado) por lo que probaremos con un linkage completo en lugar de simple:

In [ ]:
mergings_complete=linkage(crime_data,method='complete',metric='euclidean')
fig = plt.figure(figsize = (34,15))
dendrogram(mergings_complete)
plt.show()

Este dendograma tiene mucho mejor aspecto. Parece interesante cortar a la altura cuatro donde se generaría cuatro clusters, para ello empleamos el método cut_tree:

In [ ]:
from scipy.cluster.hierarchy import cut_tree
etiquetas_cluster=cut_tree(mergings_complete,n_clusters=4).reshape(-1,)
etiquetas_cluster

Una vez generadas las etiquetas de los clusters podemos asociarlas como siempre a los datos para sacar conclusiones:

In [ ]:
cluster_jerar = raw_data
cluster_jerar['ClusterID'] = etiquetas_cluster

In [ ]:
cluster_jerar.head()

Una vez hecho esto procedemos a inspeccionar cada cluster para evaluar la calidad de las agrupaciones generadas:

In [ ]:
cluster_jerar[cluster_jerar.ClusterID==0]

Parece complicado definir las características de este cluster a simple vista:

In [ ]:
cluster_jerar[cluster_jerar.ClusterID==1]

Estos podrían ser los estados más urbanizados

In [ ]:
cluster_jerar[cluster_jerar.ClusterID==2]

Estos estados también parecen difícil de agrupar en un principio.

In [ ]:
cluster_jerar[cluster_jerar.ClusterID==3]

Estos estados parecen los que menor ratio de criminalidad tienen.

Como la interpretación no ha sido sencilla vamos a calcular la media de cada variable agregada por cada cluster para intentar esclarecer más esta separación:

In [ ]:
Cluster_Murder=pd.DataFrame(cluster_jerar.groupby(["ClusterID"]).Murder.mean())
Cluster_Assault=pd.DataFrame(cluster_jerar.groupby(["ClusterID"]).Assault.mean())
Cluster_UrbanPop=pd.DataFrame(cluster_jerar.groupby(["ClusterID"]).UrbanPop.mean())
Cluster_Rape=pd.DataFrame(cluster_jerar.groupby(["ClusterID"]).Rape.mean())

In [ ]:
media_agregada_jerar = pd.concat([Cluster_Murder,Cluster_Assault,Cluster_UrbanPop,Cluster_Rape], axis=1)

In [ ]:
media_agregada_jerar

* Con esta tabla podemos esclarecer algo más la separación generada. En el cluster 1 tenemos los estados más urbanizados con mayor ratio de asaltos y violaciones. La tasa de asesinato es así mismo bastante alta.

* En el cluster 2 tenemos los estados más urbanizados con un crimen más moderado, se reducen drásticamente todos los tipos de crímenes respecto al cluster 1 mientras que la urbanización no baja tanto.

* En el cluster 0 tenemos los estados menos urbanizados con mayor criminalidad, digamos que este cluster y el 3 agrupan los estados menos urbanizados recogiendo este a aquellos con mayor criminalidad.

* Por último en el cluster 3 tenemos las tasas más bajas respecto a cualquier criminalidad así como la tasa más baja de urbanización. Son los estados menos urbanizados y más seguros.

In [ ]:
estados_poco_urbanos = cluster_jerar[cluster_jerar.ClusterID==0]

In [ ]:
estados_urbanos = cluster_jerar[cluster_jerar.ClusterID==1]

In [ ]:
estados_urbanos_seguros=cluster_jerar[cluster_jerar.ClusterID==2]

In [ ]:
estados_poco_urbanos_seguros = cluster_jerar[cluster_jerar.ClusterID==3]

Podemos apoyarnos en distintas gráficas para confirmar las conclusiones extraídas previamente:

In [ ]:
fig = plt.figure(figsize = (10,6))
media_agregada_jerar.rename(index={0: 'Estados poco urbanos'},inplace=True)
media_agregada_jerar.rename(index={1: 'Estados urbanos'},inplace=True)
media_agregada_jerar.rename(index={2: 'Estados urbanos seguros '},inplace=True)
media_agregada_jerar.rename(index={3: 'Estados poco urbanos seguros '},inplace=True)
s=sns.barplot(x=media_agregada_jerar.index,y='Murder',data=media_agregada_jerar)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Tasa de asesinatos', fontsize=10)
plt.title('Estados en base al asesinato')
plt.show()



In [ ]:
fig = plt.figure(figsize = (10,6))
s=sns.barplot(x=media_agregada_jerar.index,y='Assault',data=media_agregada_jerar)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Tasa de Asaltos', fontsize=10)
plt.title('Estados en base al asalto')
plt.show()



In [ ]:
fig = plt.figure(figsize = (10,6))
s=sns.barplot(x=media_agregada_jerar.index,y='UrbanPop',data=media_agregada_jerar)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Tasa de Población', fontsize=10)
plt.title('Estados en base a la población')
plt.show()




In [ ]:
fig = plt.figure(figsize = (10,6))
s=sns.barplot(x=media_agregada_jerar.index,y='Rape',data=media_agregada_jerar)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Tasa de Violaciones', fontsize=10)
plt.title('Estados en base a la violación')
plt.show()




Los gráficos de cajas también nos permiten confirmar estas ideas:

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='ClusterID',y='Rape',data=cluster_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Número de violaciones', fontsize=10)
plt.title('Violación según el grupo')
plt.show()

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='ClusterID',y='Assault',data=cluster_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Número de asaltos', fontsize=10)
plt.title('Asalto según el grupo')
plt.show()

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='ClusterID',y='UrbanPop',data=cluster_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Población', fontsize=10)
plt.title('Población según el grupo')
plt.show()

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='ClusterID',y='Murder',data=cluster_jerar)
plt.xlabel('Clusters', fontsize=10)
plt.ylabel('Número de asesinatos', fontsize=10)
plt.title('Asesinato según el grupo')
plt.show()

Finalmente podemos observar como se distribuyen las variables por los estados asociados a cada cluster:

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=estados_urbanos_seguros.columns[0],y='Murder',data=estados_urbanos_seguros)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Asesinatos', fontsize=10)
plt.title('Distribución de asesinatos en los estados urbanos seguros')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=estados_urbanos_seguros.columns[0],y='Murder',data=estados_urbanos)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Asesinatos', fontsize=10)
plt.title('Distribución de asesinatos en los estados urbanos')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=estados_urbanos_seguros.columns[0],y='Murder',data=estados_poco_urbanos_seguros)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Asesinatos', fontsize=10)
plt.title('Distribución de asesinatos en los estados poco urbanos seguros')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=estados_urbanos_seguros.columns[0],y='Murder',data=estados_poco_urbanos)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('Estados', fontsize=10)
plt.ylabel('Asesinatos', fontsize=10)
plt.title('Distribución de asesinatos en los estados poco urbanos')
plt.show()

## Conclusiones

En este caso ambos clusters devuelven resultados muy interesantes, frente al caso de países donde solo el método no jerárquico devolvía resultados aceptables. Aunque ambos son buenos resultados me inclinaría por los resultados del método jerárqico en este caso por dos motivos:

* Con cuatro clusters nos devuelve información no trivial como la separación de estados muy urbanizados con más y menos criminalidad mostrando que aunque el grado de urbanización está correlacionado con la criminalidad no es ni por asomo una correlación total y es posible encontrar estados urbanizados con poco crimen.

* Al ser un método jerárquico podríamos profundizar más en el estudio realizando el corte más abajo en el dendograma y obteniendo un mayor número de clusters que podrían aportar nuevas informaciones de interés.

Además el dendograma nos permite estudiar la proximidad de los estados que se encuentran en sí dentro del mismo cluster lo cual también puede resultar interesante.